In [2]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf
from collections import deque
import time
from ultralytics import YOLO
import logging
import csv
from datetime import datetime

# --- Logger helper ---
class StateLogger:
    def __init__(self, log_txt_path="log.txt", log_csv_path="log.csv"):
        self.log_txt_path = log_txt_path
        self.log_csv_path = log_csv_path

        self.logger = logging.getLogger("StateLogger")
        self.logger.setLevel(logging.INFO)
        formatter = logging.Formatter('%(asctime)s, %(levelname)s, %(message)s', datefmt='%Y-%m-%d %H:%M:%S')

        # Console
        ch = logging.StreamHandler()
        ch.setFormatter(formatter)
        self.logger.addHandler(ch)

        # Plain text file
        fh = logging.FileHandler(log_txt_path)
        fh.setFormatter(formatter)
        self.logger.addHandler(fh)

        # Create CSV header if file doesn't exist
        try:
            with open(self.log_csv_path, mode='x', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(['timestamp', 'level', 'message'])
        except FileExistsError:
            pass

    def log_event(self, message, level='INFO'):
        # Log to console + txt
        if level == 'INFO':
            self.logger.info(message)
        elif level == 'WARNING':
            self.logger.warning(message)
        elif level == 'ERROR':
            self.logger.error(message)
        else:
            self.logger.info(message)

        # Also write to CSV
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        with open(self.log_csv_path, mode='a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([timestamp, level, message])

# --- Constants ---
GAZE_MODEL_PATH = "gaze_lock_model_finetuned.keras"
OBJECT_MODEL_PATH = 'yolov8n.pt'
IMPORTANT_CLASSES = ['person', 'cell phone', 'laptop', 'book', 'tv']

# Eye landmark indices
LEFT_EYE_INDICES = [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398]
RIGHT_EYE_INDICES = [33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161, 246]

class GazeDetector:
    def __init__(self, model_path=GAZE_MODEL_PATH, buffer_size=5):
        try:
            self.model = tf.keras.models.load_model(model_path)
            print(f"Gaze model '{model_path}' loaded successfully.")
        except Exception as e:
            print(f"Error loading gaze model from '{model_path}': {e}")
            # Create a dummy model if loading fails
            input_left = tf.keras.Input(shape=(64, 64, 3), name="left_eye_input")
            input_right = tf.keras.Input(shape=(64, 64, 3), name="right_eye_input")
            dummy_output = tf.keras.layers.Dense(2, activation='softmax')(tf.keras.layers.Flatten()(input_left))
            self.model = tf.keras.Model(inputs=[input_left, input_right], outputs=dummy_output)
            print("Using a dummy gaze model as a fallback.")

        # MediaPipe initialization
        self.mp_face_mesh = mp.solutions.face_mesh
        self.face_mesh = self.mp_face_mesh.FaceMesh(
            max_num_faces=1,
            refine_landmarks=True,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )

        self.LEFT_EYE_INDICES = LEFT_EYE_INDICES
        self.RIGHT_EYE_INDICES = RIGHT_EYE_INDICES
        self.confidence_buffer = deque(maxlen=buffer_size)
        self.internal_gaze_threshold = 0.5

    def _calculate_eye_rect(self, eye_landmarks, image_width, image_height):
        x_coords = [lm.x * image_width for lm in eye_landmarks]
        y_coords = [lm.y * image_height for lm in eye_landmarks]

        if not x_coords or not y_coords:
            return (0,0,0,0)

        eye_width = max(x_coords) - min(x_coords)
        margin = int(eye_width * 0.5)
        x_min = int(min(x_coords) - margin)
        x_max = int(max(x_coords) + margin)
        y_min = int(min(y_coords) - margin)
        y_max = int(max(y_coords) + margin)

        return (x_min, y_min, x_max - x_min, y_max - y_min)

    def _extract_and_preprocess_eye(self, image, rect):
        x, y, w, h = rect
        if w <= 0 or h <= 0:
            return None

        img_h, img_w = image.shape[:2]
        x_start = max(0, x)
        y_start = max(0, y)
        x_end = min(img_w, x + w)
        y_end = min(img_h, y + h)

        if x_end <= x_start or y_end <= y_start:
            return None

        eye_img = image[y_start:y_end, x_start:x_end]
        if eye_img.size == 0:
            return None

        eye_gray = cv2.cvtColor(eye_img, cv2.COLOR_BGR2GRAY)
        eye_gray = cv2.equalizeHist(eye_gray)
        eye_img_processed = cv2.cvtColor(eye_gray, cv2.COLOR_GRAY2BGR)
        eye_img_processed = cv2.resize(eye_img_processed, (64, 64))
        eye_img_processed = cv2.cvtColor(eye_img_processed, cv2.COLOR_BGR2RGB)
        eye_img_processed = eye_img_processed.astype(np.float32) / 255.0

        return eye_img_processed

    def _predict_gaze(self, left_eye_img, right_eye_img):
        left_eye_batch = np.expand_dims(left_eye_img, axis=0)
        right_eye_batch = np.expand_dims(right_eye_img, axis=0)
        prediction = self.model.predict([left_eye_batch, right_eye_batch], verbose=0)
        confidence_looking_at_screen = prediction[0][1] 
        return confidence_looking_at_screen

    def process_frame(self, frame):
        original_frame = frame.copy()
        h, w, _ = frame.shape
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.face_mesh.process(image_rgb)

        gaze_info = {
            "raw_confidence": 0.0,
            "smoothed_confidence": 0.0,
            "is_looking": False,
            "face_detected": False,
            "left_eye_img_processed": None,
            "right_eye_img_processed": None,
            "annotated_frame": original_frame
        }

        if results.multi_face_landmarks:
            gaze_info["face_detected"] = True
            face_landmarks = results.multi_face_landmarks[0]

            left_eye_landmarks = [face_landmarks.landmark[i] for i in self.LEFT_EYE_INDICES]
            right_eye_landmarks = [face_landmarks.landmark[i] for i in self.RIGHT_EYE_INDICES]

            left_rect = self._calculate_eye_rect(left_eye_landmarks, w, h)
            right_rect = self._calculate_eye_rect(right_eye_landmarks, w, h)

            left_eye_img_processed = self._extract_and_preprocess_eye(original_frame, left_rect)
            right_eye_img_processed = self._extract_and_preprocess_eye(original_frame, right_rect)
            
            gaze_info["left_eye_img_processed"] = left_eye_img_processed
            gaze_info["right_eye_img_processed"] = right_eye_img_processed

            if left_eye_img_processed is not None and right_eye_img_processed is not None:
                raw_confidence = self._predict_gaze(left_eye_img_processed, right_eye_img_processed)
                gaze_info["raw_confidence"] = float(raw_confidence)
                self.confidence_buffer.append(raw_confidence)
                if self.confidence_buffer:
                     gaze_info["smoothed_confidence"] = float(sum(self.confidence_buffer) / len(self.confidence_buffer))
                
                gaze_info["is_looking"] = gaze_info["smoothed_confidence"] > self.internal_gaze_threshold

                status_text = f"Raw: {gaze_info['raw_confidence']:.2%} Smooth: {gaze_info['smoothed_confidence']:.2%}"
                gaze_status_text = "Looking" if gaze_info["is_looking"] else "Away"
                color = (0, 255, 0) if gaze_info["is_looking"] else (0, 0, 255)

                cv2.putText(gaze_info["annotated_frame"], status_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (230, 230, 230), 2)
                cv2.putText(gaze_info["annotated_frame"], gaze_status_text, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

                cv2.rectangle(gaze_info["annotated_frame"], (left_rect[0], left_rect[1]),
                              (left_rect[0] + left_rect[2], left_rect[1] + left_rect[3]), (0, 255, 0), 1)
                cv2.rectangle(gaze_info["annotated_frame"], (right_rect[0], right_rect[1]),
                              (right_rect[0] + right_rect[2], right_rect[1] + right_rect[3]), (0, 255, 0), 1)
                
                if gaze_info["left_eye_img_processed"] is not None:
                    debug_left_eye = (gaze_info["left_eye_img_processed"] * 255).astype(np.uint8)
                    debug_left_eye = cv2.resize(debug_left_eye, (96, 96))
                    y_offset_eye = 10
                    x_offset_left_eye = gaze_info["annotated_frame"].shape[1] - 96*2 - 20
                    if x_offset_left_eye > 0 and y_offset_eye + 96 < gaze_info["annotated_frame"].shape[0]:
                         gaze_info["annotated_frame"][y_offset_eye:y_offset_eye+96, x_offset_left_eye:x_offset_left_eye+96] = debug_left_eye

                if gaze_info["right_eye_img_processed"] is not None:
                    debug_right_eye = (gaze_info["right_eye_img_processed"] * 255).astype(np.uint8)
                    debug_right_eye = cv2.resize(debug_right_eye, (96, 96))
                    x_offset_right_eye = gaze_info["annotated_frame"].shape[1] - 96 - 10
                    if x_offset_right_eye > 0 and y_offset_eye + 96 < gaze_info["annotated_frame"].shape[0]:
                        gaze_info["annotated_frame"][y_offset_eye:y_offset_eye+96, x_offset_right_eye:x_offset_right_eye+96] = debug_right_eye
            else:
                 cv2.putText(gaze_info["annotated_frame"], "Eye crop failed", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
        else:
            cv2.putText(gaze_info["annotated_frame"], "No face detected", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            self.confidence_buffer.clear()

        return gaze_info

    # def release(self):
    #     if hasattr(self, 'face_mesh') and self.face_mesh:
    #         self.face_mesh.close()
    #     print("GazeDetector resources released.")
    def release(self):
        if hasattr(self, 'face_mesh') and self.face_mesh:
            if hasattr(self.face_mesh, '_graph') and self.face_mesh._graph is not None:
                self.face_mesh.close()
        print("GazeDetector resources released.")



class ObjectDetector:
    def __init__(self, model_path=OBJECT_MODEL_PATH, important_classes=IMPORTANT_CLASSES):
        try:
            self.model = YOLO(model_path)
            print(f"Object detection model '{model_path}' loaded successfully.")
        except Exception as e:
            print(f"Error loading object detection model: {e}")
            self.model = None
        
        self.important_classes = important_classes
        self.is_active = False
        self.last_activation_time = 0
        self.activation_cooldown = 2  # seconds
    
    def detect_objects(self, frame):
        if not self.is_active or self.model is None:
            return []
        
        results = self.model(frame)[0]
        important_objects = []
        
        for box in results.boxes:
            cls_id = int(box.cls[0])
            cls_name = results.names[cls_id]
            if cls_name in self.important_classes:
                conf = float(box.conf[0])
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                important_objects.append({
                    'class_name': cls_name,
                    'confidence': conf,
                    'box': (x1, y1, x2, y2)
                })
        
        return important_objects

    def activate(self):
        current_time = time.time()
        if not self.is_active and (current_time - self.last_activation_time > self.activation_cooldown):
            self.is_active = True
            self.last_activation_time = current_time
            print(f"[{time.strftime('%H:%M:%S')}] ObjectDetector: ACTIVATED!")
            return True
        elif self.is_active:
            print(f"[{time.strftime('%H:%M:%S')}] ObjectDetector: Already active.")
            return False
        else:
            return False

    def deactivate(self):
        if self.is_active:
            self.is_active = False
            print(f"[{time.strftime('%H:%M:%S')}] ObjectDetector: DEACTIVATED.")


class LogicModule:
    def __init__(self, 
                 gaze_model_path=GAZE_MODEL_PATH, 
                 sustained_away_threshold=2.0,  # seconds of sustained away gaze to trigger object detection
                 show_video_feed=True):
        print("Initializing LogicModule...")
        self.gaze_detector = GazeDetector(model_path=gaze_model_path)
        self.object_detector = ObjectDetector()
        self.sustained_away_threshold = sustained_away_threshold
        self.show_video_feed = show_video_feed
        self.cap = None
        self.running = False

        # Logger and previous states
        self.logger = StateLogger()
        self.prev_face_detected = None
        self.prev_gaze_status = None
        self.prev_od_status = None
        
        # For tracking sustained away gaze
        self.away_start_time = None
        self.away_duration = 0.0
        
        print("LogicModule initialized.")

    def start_monitoring(self):
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            print("Error: Could not open video camera.")
            return

        print("Starting monitoring...")
        print("System will activate object detection when sustained away gaze is detected.")
        self.running = True
        
        while self.running:
            success, frame = self.cap.read()
            if not success:
                print("Failed to grab frame or end of video stream.")
                break

            current_time = time.time()
            
            # Process gaze detection
            gaze_results = self.gaze_detector.process_frame(frame)
            
            # Update sustained away gaze tracking
            if gaze_results["face_detected"]:
                if not gaze_results["is_looking"]:
                    # Gaze is away
                    if self.away_start_time is None:
                        self.away_start_time = current_time
                    else:
                        self.away_duration = current_time - self.away_start_time
                else:
                    # Gaze is back on screen
                    self.away_start_time = None
                    self.away_duration = 0.0
                    if self.object_detector.is_active:
                        self.object_detector.deactivate()
            else:
                # No face detected
                self.away_start_time = None
                self.away_duration = 0.0
                if self.object_detector.is_active:
                    self.object_detector.deactivate()
            
            # Activate object detector if sustained away gaze threshold is reached
            if (self.away_duration >= self.sustained_away_threshold and 
                not self.object_detector.is_active):
                self.object_detector.activate()
            
            # Run object detection if active
            if self.object_detector.is_active:
                important_objects = self.object_detector.detect_objects(frame)
            else:
                important_objects = []

            # --- Log on state changes ---
            face_now = gaze_results["face_detected"]
            if face_now != self.prev_face_detected:
                if face_now:
                    self.logger.log_event(f"Face detected, Gaze confidence: {gaze_results['smoothed_confidence']:.2f}")
                else:
                    self.logger.log_event("Face lost / not detected")
                self.prev_face_detected = face_now

            gaze_now = gaze_results["is_looking"]
            if face_now and gaze_now != self.prev_gaze_status:
                status = "Looking" if gaze_now else "Away"
                self.logger.log_event(f"Gaze changed: {status}, Confidence: {gaze_results['smoothed_confidence']:.2f}")
                self.prev_gaze_status = gaze_now

            od_now = self.object_detector.is_active
            if od_now != self.prev_od_status:
                if od_now:
                    self.logger.log_event("Sustained away gaze detected, activating object detection")
                else:
                    self.logger.log_event("Object detector deactivated (Gaze back on screen)")
                self.prev_od_status = od_now

            if od_now:
                for obj in important_objects:
                    self.logger.log_event(f"Detected object: {obj['class_name']} ({obj['confidence']:.2f})")
            
            # Update display frame
            if self.show_video_feed:
                display_frame = gaze_results["annotated_frame"]
                
                # Add sustained away timer
                timer_text = f"Away Timer: {self.away_duration:.1f}/{self.sustained_away_threshold:.1f}s"
                timer_color = (0, 0, 255) if self.away_duration > 0 else (200, 200, 200)
                cv2.putText(display_frame, timer_text, (10, 90), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, timer_color, 2)
                
                # Add object detection status
                od_status = f"ObjectDetector: {'ACTIVE' if self.object_detector.is_active else 'INACTIVE'}"
                od_color = (0, 255, 0) if self.object_detector.is_active else (100, 100, 100)
                cv2.putText(display_frame, od_status, (10, display_frame.shape[0] - 20), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, od_color, 2)
                
                # Draw object detections if active
                for obj in important_objects:
                    x1, y1, x2, y2 = obj['box']
                    label = f"{obj['class_name']} {obj['confidence']:.2f}"
                    cv2.rectangle(display_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(display_frame, label, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                
                cv2.imshow("Gaze-Controlled Object Detection", display_frame)

            # Check for exit key
            key = cv2.waitKey(5) & 0xFF
            if key == 27:
                print("Escape key pressed. Stopping monitoring.")
                self.running = False
        
        self.stop_monitoring()

    def stop_monitoring(self):
        print("Stopping monitoring and releasing resources...")
        self.running = False
        if self.cap is not None:
            self.cap.release()
            self.cap = None
        if self.gaze_detector is not None:
            self.gaze_detector.release()
        cv2.destroyAllWindows()
        print("Monitoring stopped and resources released.")


# --- Main execution ---
if __name__ == "__main__":
    # Create dummy gaze model if needed
    if not tf.io.gfile.exists(GAZE_MODEL_PATH):
        print(f"Warning: Model file '{GAZE_MODEL_PATH}' not found. Creating a dummy model for testing.")
        input_left = tf.keras.Input(shape=(64, 64, 3), name="left_eye_input")
        input_right = tf.keras.Input(shape=(64, 64, 3), name="right_eye_input")
        merged = tf.keras.layers.Concatenate()([tf.keras.layers.Flatten()(input_left), 
                                                tf.keras.layers.Flatten()(input_right)])
        dense1 = tf.keras.layers.Dense(16, activation='relu')(merged)
        output = tf.keras.layers.Dense(2, activation='softmax')(dense1)
        dummy_keras_model = tf.keras.Model(inputs=[input_left, input_right], outputs=output)
        dummy_keras_model.save(GAZE_MODEL_PATH)
        print(f"Dummy model saved to '{GAZE_MODEL_PATH}'. Please replace with your actual model.")

    # Initialize and run the LogicModule
    logic_module = LogicModule(
        gaze_model_path=GAZE_MODEL_PATH,
        sustained_away_threshold=2.0,  # 2 seconds of sustained away gaze
        show_video_feed=True
    )
    try:
        logic_module.start_monitoring()
    except KeyboardInterrupt:
        print("Program interrupted by user (Ctrl+C).")
    finally:
        logic_module.stop_monitoring()

Initializing LogicModule...
Gaze model 'gaze_lock_model_finetuned.keras' loaded successfully.
Object detection model 'yolov8n.pt' loaded successfully.
LogicModule initialized.
Starting monitoring...
System will activate object detection when sustained away gaze is detected.


2025-07-26 14:45:24, INFO, Face lost / not detected
2025-07-26 14:45:24, INFO, Object detector deactivated (Gaze back on screen)
2025-07-26 14:45:30, INFO, Face detected, Gaze confidence: 0.28
2025-07-26 14:45:30, INFO, Gaze changed: Away, Confidence: 0.28


[14:45:30] ObjectDetector: ACTIVATED!

0: 480x640 1 cat, 189.6ms
Speed: 13.1ms preprocess, 189.6ms inference, 17.1ms postprocess per image at shape (1, 3, 480, 640)


2025-07-26 14:45:30, INFO, Sustained away gaze detected, activating object detection



0: 480x640 (no detections), 153.0ms
Speed: 2.1ms preprocess, 153.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 cat, 143.9ms
Speed: 2.4ms preprocess, 143.9ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 cat, 154.6ms
Speed: 2.5ms preprocess, 154.6ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 cats, 129.8ms
Speed: 2.5ms preprocess, 129.8ms inference, 2.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 cats, 136.9ms
Speed: 2.1ms preprocess, 136.9ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 127.4ms
Speed: 2.7ms preprocess, 127.4ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 cat, 143.5ms
Speed: 2.0ms preprocess, 143.5ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 170.8ms
Speed: 5.0ms preprocess, 170.8ms inference, 4.1ms postprocess per 

...


0: 480x640 1 person, 191.4ms
Speed: 4.4ms preprocess, 191.4ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)


2025-07-26 14:47:05, INFO, Detected object: person (0.77)



0: 480x640 1 person, 193.0ms
Speed: 2.4ms preprocess, 193.0ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)


2025-07-26 14:47:06, INFO, Detected object: person (0.88)


Program interrupted by user (Ctrl+C).
Stopping monitoring and releasing resources...
GazeDetector resources released.
Monitoring stopped and resources released.
